In [27]:
from pathlib import Path
import pandas as pd

base_path = Path().resolve() / 'files'

cleaned_cases = pd.read_csv(f"{base_path}/../results/all_predictions_case_descriptions_cleaned_with_collector_combined.csv")
cleaned_cases = cleaned_cases.drop_duplicates(subset="file_id", keep="first")
cleaned_cases

,file_id,rank,predicted_id,predicted_name,correct_diagnosis_id,correct_diagnosis_name,case_description,patient_hpo_terms,disease_hpo_terms,overlap_hpo_terms,new_metric,fraction_overlap,fraction_matched,exact_match,deepest_ancestor,collector,responsibleGene,reference
0,2,1,OMIM:620537,Developmental and epileptic encephalopathy 112,OMIM:607208,DRAVET SYNDROME; DRVT,"The patient, a female, who is currently 11 yea...","{'HP:0012758', 'HP:0001328', 'HP:0002133', 'HP...","{'HP:0001270', 'HP:0032712', 'HP:0002133', 'HP...","{'HP:0000750', 'HP:0002069', 'HP:0001263', 'HP...",0.682135,0.125000,0.444444,False,False,IT,SCN1A,https://onlinelibrary.wiley.com/doi/full/10.11...
20,3,1,OMIM:620655,Alfadhel syndrome,OMIM:204750,ALPHA-AMINOADIPIC AND ALPHA-KETOADIPIC ACIDURI...,"patient presented with microcephaly, mild moto...","{'HP:0010863', 'HP:0001270', 'HP:0003355', 'HP...","{'HP:0000085', 'HP:0030863', 'HP:0001762', 'HP...","{'HP:0010863', 'HP:0001270', 'HP:0000252', 'HP...",0.627590,0.161290,0.625000,False,False,IT,DHTKD1,https://omim.org/entry/204750
40,4,1,OMIM:230650,"GM1-gangliosidosis, type III",OMIM:257220,"NIEMANN-PICK DISEASE, TYPE C1; NPC1",a 43-year-old man who presented with splenomeg...,"{'HP:0003651', 'HP:0000511', 'HP:0004356', 'HP...","{'HP:0000750', 'HP:0001332', 'HP:0002506', 'HP...","{'HP:0001256', 'HP:0001744', 'HP:0003651'}",0.501384,0.103448,0.428571,False,False,IT,NPC1,https://omim.org/entry/257220?search=DISEASE&h...
42,5,1,OMIM:261630,"Hyperphenylalaninemia, bh4-deficient, C",OMIM:257220,"NIEMANN-PICK DISEASE, TYPE C1; NPC1",A 25-year-old man presented with a 14-year his...,"{'HP:0002312', 'HP:0001332', 'HP:0001249', 'HP...","{'HP:0002514', 'HP:0004923', 'HP:0001332', 'HP...","{'HP:0001332', 'HP:0001249', 'HP:0002015', 'HP...",0.403764,0.238095,0.357143,False,False,IT,NPC1,https://pubmed.ncbi.nlm.nih.gov/30119649/
47,6,1,OMIM:256550,Neuraminidase deficiency,OMIM:257220,"NIEMANN-PICK DISEASE, TYPE C1; NPC1","The proband, a 28-year-old woman, presented wi...","{'HP:0025233', 'HP:0004333', 'HP:0001922', 'HP...","{'HP:0004333', 'HP:0000943', 'HP:0000518', 'HP...","{'HP:0001922', 'HP:0001744', 'HP:0004333', 'HP...",0.400000,0.117647,0.400000,False,False,IT,NPC1,https://pubmed.ncbi.nlm.nih.gov/30119649/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10062,974,1,OMIM:613869,"Myopathy, myofibrillar, fatal infantile hypert...",OMIM:613752,S-Adenosylhomocysteine hydrolase deficiency,The proband male was born at 34 weeks through ...,"{'HP:0003236', 'HP:0011410', 'HP:0012498', 'HP...","{'HP:0003236', 'HP:0000007', 'HP:0001612', 'HP...","{'HP:0003236', 'HP:0002878', 'HP:0001612', 'HP...",0.273596,0.307692,0.235294,False,False,Marina Wasef,AHCY,PMID: 35789945
10082,975,1,OMIM:614300,Hypermethioninemia due to adenosine kinase def...,OMIM:613752,S-Adenosylhomocysteine hydrolase deficiency,The first child is a female born at 37 weeks o...,"{'HP:0003236', 'HP:0008151', 'HP:0004798', 'HP...","{'HP:0001508', 'HP:0034730', 'HP:0002059', 'HP...","{'HP:0003235', 'HP:0003236', 'HP:0008151', 'HP...",0.333333,0.117647,0.333333,False,False,Marina Wasef,AHCY,PMID: 39512434
10090,976,1,OMIM:616606,Ring chromosome 14 syndrome,OMIM:613752,S-Adenosylhomocysteine hydrolase deficiency,An 18-month-old female patient was referred to...,"{'HP:0001182', 'HP:0002160', 'HP:0002240', 'HP...","{'HP:0000750', 'HP:0002384', 'HP:0000463', 'HP...","{'HP:0000316', 'HP:0001290', 'HP:0000286', 'HP...",0.366569,0.285714,0.216216,False,False,Marina Wasef,AHCY,PMID: 39634240
10110,977,1,OMIM:614498,"Rigidity and multifocal seizure syndrome, leth...",OMIM:616586,"P5CS deficiency, cutis laxa phenotype",The patient was born to a 23-year-old G2P1L1 m...,"{'HP:0001562', 'HP:0009110', 'HP:0001511', 'HP...","{'HP:0002169', 'HP:0000023', 'HP:0002104', 'HP...","{'HP:0000252', 'HP:0000023', 'HP:0000347'}",0.402169,0.083333,0.081081,False,False,Marina Wasef,ALDH18A1,PMID: 21739576


In [28]:
clin_resp = pd.read_csv(f"{base_path}/ClinResponses_with_caseDescInformation.csv")
clin_resp_lit_cases = clin_resp[clin_resp['ReportType'] == 'literature']['caseID'].unique()
clin_resp_lit_cases

array([158, 130,  97,  49,  40,  69,  45,  70,  26, 121,  84, 189, 124,
       168, 136,  25, 147, 120, 125,  61,  81, 139, 156,   4, 154, 161,
       112, 191, 175,  18,  34,  76,  80, 169, 108, 131,  67,  12, 117,
       106,  92, 182, 148,  75])

In [29]:
clinician_cases = cleaned_cases[cleaned_cases['file_id'].isin(clin_resp_lit_cases)]
cleaned_cases = cleaned_cases[~cleaned_cases['file_id'].isin(clin_resp_lit_cases)]
cleaned_cases.index = cleaned_cases['file_id']
cleaned_cases

,file_id,rank,predicted_id,predicted_name,correct_diagnosis_id,correct_diagnosis_name,case_description,patient_hpo_terms,disease_hpo_terms,overlap_hpo_terms,new_metric,fraction_overlap,fraction_matched,exact_match,deepest_ancestor,collector,responsibleGene,reference
file_id,,,,,,,,,,,,,,,,,,
2,2,1,OMIM:620537,Developmental and epileptic encephalopathy 112,OMIM:607208,DRAVET SYNDROME; DRVT,"The patient, a female, who is currently 11 yea...","{'HP:0012758', 'HP:0001328', 'HP:0002133', 'HP...","{'HP:0001270', 'HP:0032712', 'HP:0002133', 'HP...","{'HP:0000750', 'HP:0002069', 'HP:0001263', 'HP...",0.682135,0.125000,0.444444,False,False,IT,SCN1A,https://onlinelibrary.wiley.com/doi/full/10.11...
3,3,1,OMIM:620655,Alfadhel syndrome,OMIM:204750,ALPHA-AMINOADIPIC AND ALPHA-KETOADIPIC ACIDURI...,"patient presented with microcephaly, mild moto...","{'HP:0010863', 'HP:0001270', 'HP:0003355', 'HP...","{'HP:0000085', 'HP:0030863', 'HP:0001762', 'HP...","{'HP:0010863', 'HP:0001270', 'HP:0000252', 'HP...",0.627590,0.161290,0.625000,False,False,IT,DHTKD1,https://omim.org/entry/204750
5,5,1,OMIM:261630,"Hyperphenylalaninemia, bh4-deficient, C",OMIM:257220,"NIEMANN-PICK DISEASE, TYPE C1; NPC1",A 25-year-old man presented with a 14-year his...,"{'HP:0002312', 'HP:0001332', 'HP:0001249', 'HP...","{'HP:0002514', 'HP:0004923', 'HP:0001332', 'HP...","{'HP:0001332', 'HP:0001249', 'HP:0002015', 'HP...",0.403764,0.238095,0.357143,False,False,IT,NPC1,https://pubmed.ncbi.nlm.nih.gov/30119649/
6,6,1,OMIM:256550,Neuraminidase deficiency,OMIM:257220,"NIEMANN-PICK DISEASE, TYPE C1; NPC1","The proband, a 28-year-old woman, presented wi...","{'HP:0025233', 'HP:0004333', 'HP:0001922', 'HP...","{'HP:0004333', 'HP:0000943', 'HP:0000518', 'HP...","{'HP:0001922', 'HP:0001744', 'HP:0004333', 'HP...",0.400000,0.117647,0.400000,False,False,IT,NPC1,https://pubmed.ncbi.nlm.nih.gov/30119649/
7,7,1,OMIM:117360,"Spinocerebellar ataxia 29, congenital nonprogr...",OMIM:257220,"NIEMANN-PICK DISEASE, TYPE C1; NPC1 (adult form)",This male patient had no familial or personal ...,"{'HP:0000750', 'HP:0010845', 'HP:0200136', 'HP...","{'HP:0000666', 'HP:0001270', 'HP:0002384', 'HP...","{'HP:0002071', 'HP:0001332', 'HP:0001249', 'HP...",0.520000,0.092199,0.520000,False,False,IT,NPC1,https://academic.oup.com/brain/article/130/1/1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,974,1,OMIM:613869,"Myopathy, myofibrillar, fatal infantile hypert...",OMIM:613752,S-Adenosylhomocysteine hydrolase deficiency,The proband male was born at 34 weeks through ...,"{'HP:0003236', 'HP:0011410', 'HP:0012498', 'HP...","{'HP:0003236', 'HP:0000007', 'HP:0001612', 'HP...","{'HP:0003236', 'HP:0002878', 'HP:0001612', 'HP...",0.273596,0.307692,0.235294,False,False,Marina Wasef,AHCY,PMID: 35789945
975,975,1,OMIM:614300,Hypermethioninemia due to adenosine kinase def...,OMIM:613752,S-Adenosylhomocysteine hydrolase deficiency,The first child is a female born at 37 weeks o...,"{'HP:0003236', 'HP:0008151', 'HP:0004798', 'HP...","{'HP:0001508', 'HP:0034730', 'HP:0002059', 'HP...","{'HP:0003235', 'HP:0003236', 'HP:0008151', 'HP...",0.333333,0.117647,0.333333,False,False,Marina Wasef,AHCY,PMID: 39512434
976,976,1,OMIM:616606,Ring chromosome 14 syndrome,OMIM:613752,S-Adenosylhomocysteine hydrolase deficiency,An 18-month-old female patient was referred to...,"{'HP:0001182', 'HP:0002160', 'HP:0002240', 'HP...","{'HP:0000750', 'HP:0002384', 'HP:0000463', 'HP...","{'HP:0000316', 'HP:0001290', 'HP:0000286', 'HP...",0.366569,0.285714,0.216216,False,False,Marina Wasef,AHCY,PMID: 39634240


In [30]:
import json

# Step 1: Load your JSON file
with open(base_path / "cleaned_case_description_hpos.json", "r") as f:
    case_hpos_old = json.load(f)

with open(base_path / "cleaned_case_description_hpos_new.json", "r") as f:
    case_hpos_new = json.load(f)

case_hpos = case_hpos_old | case_hpos_new
len(case_hpos)

770

In [6]:
# name = "Galactosemia, classical"
name = "WOLF-HIRSCHHORN SYNDROME"
# name = "NIEMANN-PICK DISEASE, TYPE C1; NPC1"
cases = cleaned_cases[cleaned_cases['correct_diagnosis_name'] == name]
# cases = cleaned_cases[cleaned_cases['correct_diagnosis_name'].str.contains('niemann', case=False, na=False)]
cases.index = cases['file_id']
cases

,file_id,rank,predicted_id,predicted_name,correct_diagnosis_id,correct_diagnosis_name,case_description,patient_hpo_terms,disease_hpo_terms,overlap_hpo_terms,new_metric,fraction_overlap,fraction_matched,exact_match,deepest_ancestor,collector,responsibleGene,reference
file_id,,,,,,,,,,,,,,,,,,
536,536,1,OMIM:217980,"Corpus callosum, agenesis of, with facial anom...",OMIM:194190,WOLF-HIRSCHHORN SYNDROME,Baby boy M. J. was born at 36 weeks' gestation...,"{'HP:0030864', 'HP:0000377', 'HP:0001511', 'HP...","{'HP:0000377', 'HP:0001545', 'HP:0003196', 'HP...","{'HP:0000377', 'HP:0000347', 'HP:0000047', 'HP...",0.628602,0.159091,0.333333,False,False,FKM,IC,PMID: 25137600 DOI: 10.1097/ANC.0000000000000116
537,537,1,OMIM:235730,Mowat-Wilson syndrome,OMIM:194190,WOLF-HIRSCHHORN SYNDROME,"A 13-year-old boy, the first child of a young,...","{'HP:0000174', 'HP:0006698', 'HP:0000614', 'HP...","{'HP:0000518', 'HP:0000565', 'HP:0001270', 'HP...","{'HP:0001631', 'HP:0000047', 'HP:0001270', 'HP...",0.449221,0.117647,0.444444,False,True,FKM,IC,PMID: 34572183 PMCID: PMC8471045 DOI: 10.3390/...
538,538,1,OMIM:147920,Kabuki Syndrome 1,OMIM:194190,WOLF-HIRSCHHORN SYNDROME,"A 4-year-old girl, the only child of non-relat...","{'HP:0000164', 'HP:0001999', 'HP:0005268', 'HP...","{'HP:0000851', 'HP:0000358', 'HP:0003196', 'HP...","{'HP:0000164', 'HP:0001631', 'HP:0000960', 'HP...",0.594731,0.054645,0.588235,False,False,FKM,IC,PMID: 34572183 PMCID: PMC8471045 DOI: 10.3390/...
539,539,1,OMIM:216340,Yunis-Varon syndrome,OMIM:194190,WOLF-HIRSCHHORN SYNDROME,"A 2-year-old boy, the second child of an unrel...","{'HP:0007598', 'HP:0001156', 'HP:0001511', 'HP...","{'HP:0007333', 'HP:0005461', 'HP:0000737', 'HP...","{'HP:0007598', 'HP:0001511', 'HP:0000047', 'HP...",0.573786,0.081818,0.562500,False,False,FKM,IC,PMID: 34572183 PMCID: PMC8471045 DOI: 10.3390/...
541,541,1,OMIM:609029,Emanuel syndrome,OMIM:194190,WOLF-HIRSCHHORN SYNDROME,"A 1-year-old boy, the only child of an unrelat...","{'HP:0001344', 'HP:0001999', 'HP:0001631', 'HP...","{'HP:0001562', 'HP:0001511', 'HP:0000400', 'HP...","{'HP:0001631', 'HP:0002205', 'HP:0000960', 'HP...",0.614114,0.088235,0.600000,False,False,FKM,IC,PMID: 34572183 PMCID: PMC8471045 DOI: 10.3390/...
542,542,1,OMIM:614866,Peroxisome biogenesis disorder 5A (Zellweger),OMIM:194190,WOLF-HIRSCHHORN SYNDROME,"A 5-year-old girl, the first child of an unrel...","{'HP:0001999', 'HP:0001511', 'HP:0001631', 'HP...","{'HP:0000311', 'HP:0001433', 'HP:0001762', 'HP...","{'HP:0001511', 'HP:0001631', 'HP:0001252', 'HP...",0.440545,0.080460,0.437500,False,False,FKM,IC,PMID: 34572183 PMCID: PMC8471045 DOI: 10.3390/...
544,544,1,OMIM:618356,Neurodevelopmental disorder with central and p...,OMIM:194190,WOLF-HIRSCHHORN SYNDROME,A female patient was observed with significant...,"{'HP:0000316', 'HP:0001511', 'HP:0000347', 'HP...","{'HP:0000878', 'HP:0002075', 'HP:0000237', 'HP...","{'HP:0000316', 'HP:0011968', 'HP:0000347', 'HP...",0.700000,0.189189,0.700000,False,False,FKM,IC,PMID: 10103318 DOI: 10.1542/peds.103.4.830
545,545,1,OMIM:615802,Neurodevelopmental disorder with dysmorphic fe...,OMIM:194190,WOLF-HIRSCHHORN SYNDROME,This female patient had notable intrauterine g...,"{'HP:0001511', 'HP:0003202', 'HP:0001252', 'HP...","{'HP:0000582', 'HP:0000395', 'HP:0000400', 'HP...","{'HP:0003202', 'HP:0001252', 'HP:0000733', 'HP...",0.745815,0.118644,0.700000,False,False,FKM,IC,PMID: 10103318 DOI: 10.1542/peds.103.4.830
546,546,1,OMIM:264470,Peroxisomal acyl-CoA oxidase deficiency,OMIM:194190,WOLF-HIRSCHHORN SYNDROME,A female patient presented with severe develop...,"{'HP:0008619', 'HP:0002839', 'HP:0010864', 'HP...","{'HP:0000654', 'HP:0000737', 'HP:0002415', 'HP...","{'HP:0010864', 'HP:0008619', 'HP:0011344'}",0.600000,0.078947,0.600000,False,False,FKM,IC,PMID: 10103318 DOI: 10.1542/peds.103.4.830


In [7]:
cleaned_cases = cleaned_cases.drop(cases.index)

In [8]:
cases['file_id'] = cases['file_id'].astype(str)
cases['HPO_terms'] = cases['file_id'].map(case_hpos)
cleaned_cases['file_id'] = cleaned_cases['file_id'].astype(str)
cleaned_cases['HPO_terms'] = cleaned_cases['file_id'].map(case_hpos)
len(cases.dropna()), len(cleaned_cases.dropna())

/var/folders/wm/nmccdgwx5v33gkxvs2l3pgvc0000gn/T/ipykernel_78638/4212169491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['file_id'] = cases['file_id'].astype(str)
/var/folders/wm/nmccdgwx5v33gkxvs2l3pgvc0000gn/T/ipykernel_78638/4212169491.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['HPO_terms'] = cases['file_id'].map(case_hpos)


(20, 708)

In [10]:
cases["correct_diagnosis_name"] = cases["correct_diagnosis_name"].str.replace(name, f"{name} atypical")
cleaned_cases["correct_diagnosis_name"] = (cleaned_cases["correct_diagnosis_name"] + " atypical")

/var/folders/wm/nmccdgwx5v33gkxvs2l3pgvc0000gn/T/ipykernel_78638/1929001389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases["correct_diagnosis_name"] = cases["correct_diagnosis_name"].str.replace(name, f"{name} atypical")


In [11]:
extra_diseases = {}
for disease_name in cleaned_cases['correct_diagnosis_name'].unique():
    sub_df = cleaned_cases[cleaned_cases['correct_diagnosis_name'] == disease_name]
    atypical_terms = set(t for sublist in sub_df["HPO_terms"] for t in sublist)
    extra_diseases[disease_name] = atypical_terms

In [22]:
# Random states 42, 64, 128
cases_sample = cases.sample(10, random_state=64)
cases_remaining = cases.drop(cases_sample.index)

In [23]:
from utils.synthetic_dataset_utils import get_disease_profile

atypical_terms = set(t for sublist in cases_sample["HPO_terms"] for t in sublist)

cases_remaining["atypical_terms"] = [list(atypical_terms)] * len(cases_remaining)

cases_remaining["overlap_count_atypical"] = cases_remaining["HPO_terms"].apply(
    lambda terms: len(set(terms) & atypical_terms)
)

hpo_data = pd.read_csv(base_path / 'phenotype.hpoa', delimiter='\t', comment='#', low_memory=False)

omim = cases_remaining.iloc[0]['correct_diagnosis_id']
classical_hpos = set(get_disease_profile(hpo_data, disease_id=omim).keys())

cases_remaining["overlap_count_classical"] = cases_remaining["HPO_terms"].apply(
    lambda terms: len(set(terms) & classical_hpos)
)

cases_remaining["overlap_terms_atypical"] = cases_remaining["HPO_terms"].apply(
    lambda terms: list(set(terms) & atypical_terms)
)

# cases_remaining.drop(columns=["HPO_terms"], inplace=True)

cases_remaining

,file_id,rank,predicted_id,predicted_name,correct_diagnosis_id,correct_diagnosis_name,case_description,patient_hpo_terms,disease_hpo_terms,overlap_hpo_terms,...,exact_match,deepest_ancestor,collector,responsibleGene,reference,HPO_terms,atypical_terms,overlap_count_atypical,overlap_count_classical,overlap_terms_atypical
file_id,,,,,,,,,,,,,,,,,,,,,
538,538,1,OMIM:147920,Kabuki Syndrome 1,OMIM:194190,WOLF-HIRSCHHORN SYNDROME atypical,"A 4-year-old girl, the only child of non-relat...","{'HP:0000164', 'HP:0001999', 'HP:0005268', 'HP...","{'HP:0000851', 'HP:0000358', 'HP:0003196', 'HP...","{'HP:0000164', 'HP:0001631', 'HP:0000960', 'HP...",...,False,False,FKM,IC,PMID: 34572183 PMCID: PMC8471045 DOI: 10.3390/...,"[HP:0000164, HP:0000252, HP:0000519, HP:000096...","[HP:0000967, HP:0001629, HP:0001156, HP:041003...",10,9,"[HP:0001629, HP:0004322, HP:0001631, HP:000126..."
541,541,1,OMIM:609029,Emanuel syndrome,OMIM:194190,WOLF-HIRSCHHORN SYNDROME atypical,"A 1-year-old boy, the only child of an unrelat...","{'HP:0001344', 'HP:0001999', 'HP:0001631', 'HP...","{'HP:0001562', 'HP:0001511', 'HP:0000400', 'HP...","{'HP:0001631', 'HP:0002205', 'HP:0000960', 'HP...",...,False,False,FKM,IC,PMID: 34572183 PMCID: PMC8471045 DOI: 10.3390/...,"[HP:0000252, HP:0000750, HP:0000960, HP:000125...","[HP:0000967, HP:0001629, HP:0001156, HP:041003...",9,4,"[HP:0001631, HP:0000960, HP:0001250, HP:000075..."
544,544,1,OMIM:618356,Neurodevelopmental disorder with central and p...,OMIM:194190,WOLF-HIRSCHHORN SYNDROME atypical,A female patient was observed with significant...,"{'HP:0000316', 'HP:0001511', 'HP:0000347', 'HP...","{'HP:0000878', 'HP:0002075', 'HP:0000237', 'HP...","{'HP:0000316', 'HP:0011968', 'HP:0000347', 'HP...",...,False,False,FKM,IC,PMID: 10103318 DOI: 10.1542/peds.103.4.830,"[HP:0000252, HP:0000316, HP:0000347, HP:000125...","[HP:0000967, HP:0001629, HP:0001156, HP:041003...",5,6,"[HP:0011968, HP:0001511, HP:0000347, HP:000125..."
546,546,1,OMIM:264470,Peroxisomal acyl-CoA oxidase deficiency,OMIM:194190,WOLF-HIRSCHHORN SYNDROME atypical,A female patient presented with severe develop...,"{'HP:0008619', 'HP:0002839', 'HP:0010864', 'HP...","{'HP:0000654', 'HP:0000737', 'HP:0002415', 'HP...","{'HP:0010864', 'HP:0008619', 'HP:0011344'}",...,False,False,FKM,IC,PMID: 10103318 DOI: 10.1542/peds.103.4.830,"[HP:0002839, HP:0008619, HP:0010864, HP:001134...","[HP:0000967, HP:0001629, HP:0001156, HP:041003...",2,1,"[HP:0010864, HP:0011344]"
548,548,1,OMIM:620465,"Epilepsy, early-onset, 3, with or without deve...",OMIM:194190,WOLF-HIRSCHHORN SYNDROME atypical,A female patient exhibited a characteristic cr...,"{'HP:0001344', 'HP:0000316', 'HP:0001631', 'HP...","{'HP:0007359', 'HP:0002384', 'HP:0000006', 'HP...","{'HP:0001344', 'HP:0010864'}",...,False,False,FKM,IC,PMID: 10103318 DOI: 10.1542/peds.103.4.830,"[HP:0000316, HP:0000426, HP:0001250, HP:000134...","[HP:0000967, HP:0001629, HP:0001156, HP:041003...",5,4,"[HP:0010864, HP:0001631, HP:0001250, HP:001134..."
552,552,1,OMIM:613385,"Autoimmune disease, multisystem, with facial d...",OMIM:194190,WOLF-HIRSCHHORN SYNDROME atypical,A male patient had profound developmental dela...,"{'HP:0002353', 'HP:0002205', 'HP:0001290', 'HP...","{'HP:0000331', 'HP:0000358', 'HP:0001270', 'HP...","{'HP:0001999', 'HP:0001252', 'HP:0001290', 'HP...",...,False,True,FKM,IC,PMID: 10103318 DOI: 10.1542/peds.103.4.830,"[HP:0001252, HP:0001290, HP:0001999, HP:000220...","[HP:0000967, HP:0001629, HP:0001156, HP:041003...",6,2,"[HP:0012736, HP:0001252, HP:0002353, HP:000220..."
553,553,1,OMIM:616462,"Acrofacial dysostosis, Cincinnati type",OMIM:194190,WOLF-HIRSCHHORN SYNDROME atypical,"This female patient exhibited scoliosis, kypho...","{'HP:0000924', 'HP:0001166', 'HP:0001155', 'HP...","{'HP:0009110', 'HP:0008807', 'HP:0001511', 'HP...","{'HP:0002650', 'HP:0001250'}",...,False,False,FKM,IC,PMID: 10103318 DOI: 10.1542/peds.103.4.830,"[HP:0000924, HP:0001155, HP:0001166, HP:000125...","[HP:0000967, HP:0001629, HP:0001156, HP:0

In [24]:
from utils.hpo_ontology import load_ontology

atypical = {f'{name} atypical': atypical_terms}
extra_diseases = extra_diseases | atypical

# Load the HPO ontology
graph, ic_dict = load_ontology(annotations='OMIM')

print(f"{name} cases:")
for i in range(len(cases_remaining)):

    patient_hpos     = cases_remaining.iloc[i]['HPO_terms']  # list of lists of HPO IDs
    patient_idx      = cases_remaining.iloc[i]['file_id']

    pred_df = predict_disease_from_hpo(patient_hpos,
                             graph,
                             ic_dict,
                             OMIM_or_ORPHA='OMIM',
                             semantic_similarity=True,
                             pot_diseases=None,
                             weighted_score_active=True,
                             neg_hpos=None,  # Negative HPO parameter
                             extra_diseases=extra_diseases
                             )
    
    row_num_atypical = pred_df.reset_index(drop=True).index[pred_df["ID"] == f"{name} atypical"]
    row_num_classical = pred_df.reset_index(drop=True).index[pred_df["ID"] == omim]

    if not row_num_atypical.empty:
        if row_num_classical.empty:
            print(f"Patient {patient_idx:<3} | classical → atypical rank : "
                f"{'Not Found':>10} → {row_num_atypical[0] + 1:<3}")
        else:
            print(f"Patient {patient_idx:<3} | classical → atypical rank : "
                f"{row_num_classical[0] + 1:<3} → {row_num_atypical[0] + 1:<3}")
    else:
        print(f"Patient {patient_idx:<3} | classical → atypical rank : "
            f"{'Value not found':>20}")


WOLF-HIRSCHHORN SYNDROME cases:
Patient 538 | classical → atypical rank : 6   → 1  
Patient 541 | classical → atypical rank : 161 → 1  
Patient 544 | classical → atypical rank : 8   → 62 
Patient 546 | classical → atypical rank : 384 → 53 
Patient 548 | classical → atypical rank : 57  → 6  
Patient 552 | classical → atypical rank : 440 → 1  
Patient 553 | classical → atypical rank : 60  → 329
Patient 554 | classical → atypical rank : 32  → 5  
Patient 555 | classical → atypical rank : 121 → 1  
Patient 557 | classical → atypical rank : 1168 → 39 
